In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key

HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1048724
2024-09-20 16:17:39,777 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "nigeria", "city": "abuja", "street": "us-embassy", "aqicn_url": "https://api.waqi.info/feed/@13449", "latitude": 9.6, "longitude": 8.0}'

In [4]:
air_quality_fg = fs.get_feature_group(
    name = "air_quality",
    version = 1,
)

weather_fg = fs.get_feature_group(
    name = "weather",
    version = 1,
)

In [5]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df['pm25'] = aq_today_df['pm25'].astype(float)

In [6]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float64       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 180.0+ bytes


In [7]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

hourly_df

Coordinates 9.5°N 8.0°E
Elevation 737.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant
date,,,,
2024-09-20 00:00:00,20.450001,0.0,4.198285,300.963684
2024-09-20 01:00:00,20.250000,0.0,4.610250,308.659821
2024-09-20 02:00:00,20.150000,0.0,5.091168,315.000092
2024-09-20 03:00:00,20.150000,0.0,5.116561,320.710602
2024-09-20 04:00:00,20.100000,0.0,5.315336,331.699341
...,...,...,...,...
2024-09-29 19:00:00,22.250000,0.5,3.545589,156.037506
2024-09-29 20:00:00,21.799999,0.5,3.706427,150.945496
2024-09-29 21:00:00,21.400000,0.5,3.893995,146.309906


In [8]:
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-09-20,28.600000,0.1,5.014219,338.962402,abuja
1,2024-09-21,28.799999,0.1,10.703569,19.653913,abuja
2,2024-09-22,26.350000,0.2,15.941944,173.517029,abuja
3,2024-09-23,26.250000,0.1,5.959060,205.016861,abuja
4,2024-09-24,28.600000,0.0,5.447788,352.405426,abuja
5,2024-09-25,28.950001,0.0,3.545589,156.037506,abuja
6,2024-09-26,27.100000,0.1,9.826088,151.557175,abuja
7,2024-09-27,29.100000,0.1,4.198285,239.036301,abuja
8,2024-09-28,28.350000,0.1,5.804825,330.255219,abuja
9,2024-09-29,27.650000,0.0,4.680000,180.000000,abuja


In [9]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 452.0+ bytes


In [10]:
aq_today_df

,pm25,country,city,street,date,url
0,57.0,nigeria,abuja,us-embassy,2024-09-20,https://api.waqi.info/feed/@13449


In [11]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float64       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 180.0+ bytes


In [12]:
air_quality_fg.insert(aq_today_df)

2024-09-20 16:17:56,638 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1048724/fs/1040451/fg/1204590


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1048724/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1d5fd0101a0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 621569
         }
       },
       "result": {
         "observed_value": 57.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-09-20T03:17:56.000638Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "unsu

In [13]:
weather_fg.insert(daily_df)

2024-09-20 16:18:15,215 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1048724/fs/1040451/fg/1206605


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1048724/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1d5fcfb23c0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 622594
         }
       },
       "result": {
         "observed_value": 3.545588731765747,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-09-20T03:18:15.000215Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_